<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/neo4jdocs/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install selenium==4.7.2 transformers==4.25.1 sentence-transformers===2.2.2 graphdatascience

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!apt-get update
!apt install chromium-chromedriver
!apt install -y xvfb

!pip install undetected-chromedriver==3.1.7
!pip install PyVirtualDisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [7]:
# Only run this if you use Google Colab

!zip -j /content/chromedriver_linux64.zip /usr/bin/chromedriver
# replace python3.8 with your own version of python in case it's not the same
patcher_src = (
    "/usr/local/lib/python3.8/dist-packages/undetected_chromedriver/patcher.py"
)
with open(patcher_src, "r") as f:
    contents = f.read()
    contents = contents.replace(
        "return urlretrieve(u)[0]",
        "return urlretrieve('file:///content/chromedriver_linux64.zip',"
        "filename='/tmp/chromedriver_linux64.zip')[0]",
    )
with open(patcher_src, "w") as f:
    f.write(contents)

updating: chromedriver (deflated 51%)


In [8]:
# Only run this if you use Google Colab
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
import undetected_chromedriver.v2 as uc
from pyvirtualdisplay import Display

display = Display(visible=0, size=(800, 600))
display.start()

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
wd = uc.Chrome(options=options)

In [10]:
def extract_text_by_class(c):
    global wd
    try:
        content = wd.find_element(By.CLASS_NAME, c)
        return content.text
    except:
        return ""


def extract_links_by_xpath(xpath):
    global wd
    links = set()
    try:
        a_elems = wd.find_elements(By.XPATH, xpath)
        for elem in a_elems:
            link = elem.get_attribute("href")
            if link == "javascript:void(0)":
                continue
            # Remove links to images and various files
            if (
                link.endswith(".png")
                or link.endswith(".json")
                or link.endswith(".txt")
                or link.endswith(".svg")
                or link.endswith(".ipynb")
                or link.endswith(".jpg")
                or link.endswith(".pdf")
                or link.endswith(".mp4")
                or "mailto" in link
                or len(link) > 300
            ):
                continue
            # Remove anchors
            link = link.split("#")[0]
            # Remove parameters
            link = link.split("?")[0]
            # Remove trailing forward slash
            link = link.rstrip("/")
            links.add(link)
        return list(links)
    except:
        return []

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModel
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/bert-uncased-keyword-extractor")
model = AutoModelForTokenClassification.from_pretrained(
    "yanekyuk/bert-uncased-keyword-extractor"
)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


def extract_keywords(text):
    """
    Extract keywords and construct them back from tokens
    """
    result = list()
    keyword = ""
    for token in nlp(text):
        if token['entity'] == 'I-KEY':
            keyword += token['word'][2:] if \
              token['word'].startswith("##") else f" {token['word']}"
        else:
            if keyword:
                result.append(keyword)
            keyword = token['word']
    # Add the last keyword
    result.append(keyword)
    return list(set(result))

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [12]:
extract_keywords(
    """
Broadcom agreed to acquire cloud computing company VMware in a $61 billion (€57bn) cash-and stock deal.
"""
)

['cloud computing', 'broadcom', 'vmware']

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def generate_embeddings(text):
    embeddings = model.encode(text)
    return [float(x) for x in embeddings.tolist()]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [14]:
generate_embeddings(
    """
Web APIs are a huge opportunity to access and integrate data from any sources with your graph. Most of them provide the data in JSON format.

The Load JSON procedures retrieve data from URLs or maps and turn it into map value(s) for Cypher to consume. Cypher has support for deconstructing nested documents with dot syntax, slices, UNWIND etc. so it is easy to turn nested data into graphs.

Sources with multiple JSON objects (JSONL,JSON Lines) in a stream, like the streaming Twitter format or the Yelp Kaggle dataset, are also supported,
"""
)

[-0.06911396980285645,
 0.05224503576755524,
 -0.007201760541647673,
 -0.015374782495200634,
 0.0039060856215655804,
 -0.050424616783857346,
 -0.10675833374261856,
 0.06225704401731491,
 0.0032547670416533947,
 0.013045662082731724,
 0.007432159036397934,
 0.002438636962324381,
 0.07324249297380447,
 0.060171280056238174,
 0.13454291224479675,
 0.03666401281952858,
 -0.007088623009622097,
 0.030180802568793297,
 0.026259925216436386,
 -0.14783154428005219,
 0.011475381441414356,
 0.0016846637008711696,
 -0.04652782529592514,
 -0.03765556216239929,
 0.048804279416799545,
 0.08559443056583405,
 -0.030265966430306435,
 0.052127689123153687,
 0.045675404369831085,
 0.06373853981494904,
 -0.029776575043797493,
 -0.010671338066458702,
 -0.05804384499788284,
 0.05813639611005783,
 -0.09820526838302612,
 0.009667903184890747,
 0.03530018404126167,
 0.00953015498816967,
 0.01339051965624094,
 0.0526607520878315,
 0.04211463779211044,
 0.01219760812819004,
 -0.04744797945022583,
 0.0182718448340

In [ ]:
from selenium.webdriver.common.by import By

entry_url = "https://neo4j.com/docs"
data = dict()
visit_list = [entry_url]
already_visited = []

while visit_list:
    # Visit the URL
    current_url = visit_list.pop()
    if current_url in already_visited:
        continue
    print(current_url)
    try:
        wd.get(current_url)
    except:
        print(f"Couldn't open {current_url}")
        already_visited.append(current_url)
        continue
    # If redirect
    try:
        actual_url = wd.current_url.rtrim("/")
        if actual_url != current_url:
            # Store the redirect information
            data[current_url] = {
              "links": [],
              "text": None,
              "embeddings": [],
              "keywords": [],
              "redirects": [actual_url]
            }
            already_visited.append(current_url)
            # Content is being extracted from the actual url
            current_url = actual_url
    except:
        pass
    # Extract text from the content div
    text = extract_text_by_class("content")
    # If nothing is found, try article div
    if not text:
        text = extract_text_by_class("article")
    # If nothing is found, try page div
    if not text:
        text = extract_text_by_class("page")
    if not text:
        text = extract_text_by_class("single-user-story")
    # Check if 404
    try:
        if ("Sorry, page not found" in wd.find_element(By.TAG_NAME, "body").text):
            text = "404"
    except:
        pass

    # Generate paragraph embedding & extract keywords
    if text:
        embeddings = generate_embeddings(text)
        keywords = extract_keywords(text)
    else:
        embeddings = []
        keywords = []

    # Extract links from the content div
    links = extract_links_by_xpath("//div[@class='content']//a[@href]")
    # If nothing is found, try article div
    if not links:
        links = extract_links_by_xpath("//article[@class='article']//a[@href]")
    if not links:
        links = extract_links_by_xpath("//article//a[@href]")

    if not links and not text:
        print(f"Couldn't retrieve the data from {current_url}")
    # Store page information
    data[current_url] = {
        "links": links,
        "text": text,
        "embeddings": embeddings,
        "keywords": keywords,
        "redirects": []
    }
    # Crawling information
    already_visited.append(current_url)
    # Don't leave neo4j.com while crawling
    # and avoid community and sandbox
    visit_list.extend(
        [l for l in list(links) if ("neo4j.com" in l)  
        and (not l in already_visited)  
        and (not "community.neo4j.com" in l)  
        and (not "sandbox.neo4j.com" in l)]
  )

Streaming output truncated to the last 5000 lines.
https://neo4j.com/docs/cypher-manual/4.4/functions/load-csv
https://neo4j.com/docs/cypher-manual/4.4/functions/user-defined
https://neo4j.com/docs/java-reference/4.4/extending-neo4j/aggregation-functions
https://neo4j.com/docs/java-reference/4.4/extending-neo4j/functions
https://neo4j.com/docs/java-reference/4.4/javadocs/org/neo4j/procedure/UserFunction.html
Couldn't retrieve the data from https://neo4j.com/docs/java-reference/4.4/javadocs/org/neo4j/procedure/UserFunction.html
https://neo4j.com/docs/java-reference/4.4/extending-neo4j/procedures
https://neo4j.com/docs/cypher-manual/4.4/clauses/call
https://neo4j.com/docs/cypher-manual/4.4/clauses/call-subquery
https://neo4j.com/docs/cypher-manual/4.4/introduction/transactions
https://neo4j.com/docs/cypher-manual/4.4/introduction/uniqueness
https://neo4j.com/docs/cypher-manual/4.4/query-tuning/using
https://neo4j.com/docs/cypher-manual/4.4/query-tuning/advanced-example
https://neo4j.com/

Couldn't open https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/index/lucene/QueryContext.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/index/lucene/ValueContext.html
Couldn't open https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/index/lucene/ValueContext.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/index/IndexHits.html
Couldn't open https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/graphdb/index/IndexHits.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/helpers/TransactionTemplate.html
Couldn't open https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/helpers/TransactionTemplate.html
https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/procedure/UserAggregationFunction.html
Couldn't open https://neo4j.com/docs/java-reference/3.5/javadocs/org/neo4j/procedure/UserAggregationFunction.html
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/migration/surface-cha

Couldn't open https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/schema/package-summary.html
https://neo4j.com/docs/java-reference/4.0/javadocs/com/neo4j/backup/package-summary.html
Couldn't open https://neo4j.com/docs/java-reference/4.0/javadocs/com/neo4j/backup/package-summary.html
https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/event/package-summary.html
Couldn't open https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/event/package-summary.html
https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/Entity.html
Couldn't open https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/Entity.html
https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/Transaction.html
Couldn't open https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/graphdb/Transaction.html
https://neo4j.com/docs/java-reference/4.0/javadocs/org/neo4j/dbms/api/package-summary.html
Couldn't open https://neo4j.com/docs/j

Couldn't open https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-4.3/causal-cluster
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-4.3/troubleshooting
Couldn't open https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-4.3/troubleshooting
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-4.4/standalone
Couldn't open https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-4.4/standalone
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-patch/causal-cluster
Couldn't open https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-patch/causal-cluster
https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-patch/upgrade-checklist
Couldn't open https://neo4j.com/docs/upgrade-migration-guide/current/version-4/upgrade/upgrade-patch/upgrade-checklist
https://neo4j.com/docs

Couldn't open https://neo4j.com/docs/migration-guide/4.0
https://neo4j.com/docs/driver-manual/4.1/terminology
Couldn't open https://neo4j.com/docs/driver-manual/4.1/terminology
https://neo4j.com/docs/http-api/4.1
Couldn't open https://neo4j.com/docs/http-api/4.1
https://neo4j.com/docs/status-codes/4.1
Couldn't open https://neo4j.com/docs/status-codes/4.1
https://neo4j.com/docs/driver-manual/4.1/session-api
Couldn't open https://neo4j.com/docs/driver-manual/4.1/session-api
https://neo4j.com/docs/cypher-manual/4.1/query-tuning/how-do-i-profile-a-query
Couldn't open https://neo4j.com/docs/cypher-manual/4.1/query-tuning/how-do-i-profile-a-query
https://neo4j.com/docs/cypher-manual/4.1/introduction/neo4j-databases-graphs
Couldn't open https://neo4j.com/docs/cypher-manual/4.1/introduction/neo4j-databases-graphs
https://neo4j.com/docs/http-api/4.1/actions
Couldn't open https://neo4j.com/docs/http-api/4.1/actions
https://neo4j.com/docs/cypher-manual/4.1/clauses
Couldn't open https://neo4j.com/

...

Couldn't open https://neo4j.com/developer/kb/tags/jdk
https://neo4j.com/developer/kb/protecting-against-ssrf
Couldn't open https://neo4j.com/developer/kb/protecting-against-ssrf
https://neo4j.com/developer/kb/explanation-of-data-log-console-log-error-of-tls-certificate
Couldn't open https://neo4j.com/developer/kb/explanation-of-data-log-console-log-error-of-tls-certificate
https://neo4j.com/developer/kb/protecting-against-cypher-injection
Couldn't open https://neo4j.com/developer/kb/protecting-against-cypher-injection
https://neo4j.com/developer/kb/a-lightweight-approach-to-testing-the-neo4j-rest-api-with-authentication
Couldn't open https://neo4j.com/developer/kb/a-lightweight-approach-to-testing-the-neo4j-rest-api-with-authentication
https://neo4j.com/developer/kb/tags/authentication
Couldn't open https://neo4j.com/developer/kb/tags/authentication
https://neo4j.com/developer/kb/tags/cypher-shell
Couldn't open https://neo4j.com/developer/kb/tags/cypher-shell
https://neo4j.com/develope

Couldn't open https://neo4j.com/developer/kb/tags/cluster
https://neo4j.com/developer/kb/tags/python
Couldn't open https://neo4j.com/developer/kb/tags/python
https://neo4j.com/developer/kb/how-do-i-display-the-rest-code-from-the-3-0-browser
Couldn't open https://neo4j.com/developer/kb/how-do-i-display-the-rest-code-from-the-3-0-browser
https://neo4j.com/developer/kb/explanation-of-error-failed-to-obtain-connection-towards-write-server
Couldn't open https://neo4j.com/developer/kb/explanation-of-error-failed-to-obtain-connection-towards-write-server
https://neo4j.com/developer/kb/linkurious-bolt-configuration
Couldn't open https://neo4j.com/developer/kb/linkurious-bolt-configuration
https://neo4j.com/developer/kb/tags/exception
Couldn't open https://neo4j.com/developer/kb/tags/exception
https://neo4j.com/developer/kb/tags/linkurious
Couldn't open https://neo4j.com/developer/kb/tags/linkurious
https://neo4j.com/developer/kb/tags/websocket
Couldn't open https://neo4j.com/developer/kb/tags/

Couldn't open http://browser.neo4j.com
https://neo4j.com/developer/kb/tags/chrome
Couldn't open https://neo4j.com/developer/kb/tags/chrome
https://neo4j.com/developer/kb/list-of-restricted-ports-in-browsers
Couldn't open https://neo4j.com/developer/kb/list-of-restricted-ports-in-browsers
https://neo4j.com/developer/kb/a-light-weight-approach-to-validating-network-port-connectivity
Couldn't open https://neo4j.com/developer/kb/a-light-weight-approach-to-validating-network-port-connectivity
https://neo4j.com/developer/kb/how-do-i-override-browser-configuration-settings
Couldn't open https://neo4j.com/developer/kb/how-do-i-override-browser-configuration-settings
https://neo4j.com/developer/kb/explanation-of-error-security-error-18-when-using-internet-explorer-and-neo4j-browser
Couldn't open https://neo4j.com/developer/kb/explanation-of-error-security-error-18-when-using-internet-explorer-and-neo4j-browser
https://neo4j.com/developer/kb/how-do-i-zoom-in-out-within-the-graph-visualization-of

Couldn't open https://neo4j.com/developer/kb/tags/shared
https://neo4j.com/developer/kb/tags/delete
Couldn't open https://neo4j.com/developer/kb/tags/delete
https://neo4j.com/developer/kb/neo4j-behaviour-when-running-out-of-disk-space-3-4
Couldn't open https://neo4j.com/developer/kb/neo4j-behaviour-when-running-out-of-disk-space-3-4
https://neo4j.com/developer/kb/tags/lock
Couldn't open https://neo4j.com/developer/kb/tags/lock
https://neo4j.com/developer/kb/tags/timeout
Couldn't open https://neo4j.com/developer/kb/tags/timeout
https://neo4j.com/developer/kb/explanation-of-debug-log-message-of-commits-found-after-last-check-point
Couldn't open https://neo4j.com/developer/kb/explanation-of-debug-log-message-of-commits-found-after-last-check-point
https://neo4j.com/developer/kb/tags/exclusive
Couldn't open https://neo4j.com/developer/kb/tags/exclusive
https://neo4j.com/developer/kb/tags/disk
Couldn't open https://neo4j.com/developer/kb/tags/disk
https://neo4j.com/developer/kb/tags/disk-sp

Couldn't open https://neo4j.com/developer/kb/tags/cache
https://neo4j.com/developer/kb/long-pauses-caused-by-application-code-calling-system-gc
Couldn't open https://neo4j.com/developer/kb/long-pauses-caused-by-application-code-calling-system-gc
https://neo4j.com/developer/kb/how-to-configure-off-heap-transaction-state
Couldn't open https://neo4j.com/developer/kb/how-to-configure-off-heap-transaction-state
https://neo4j.com/developer/kb/tags/error
Couldn't open https://neo4j.com/developer/kb/tags/error
https://neo4j.com/developer/kb/hosting-multiple-neo4j-instances-on-one-machine
Couldn't open https://neo4j.com/developer/kb/hosting-multiple-neo4j-instances-on-one-machine
https://neo4j.com/developer/kb/displaying-cpu-utilization-and-allocated-bytes-in-query-log
Couldn't open https://neo4j.com/developer/kb/displaying-cpu-utilization-and-allocated-bytes-in-query-log
https://neo4j.com/developer/kb/tags/query.log
Couldn't open https://neo4j.com/developer/kb/tags/query.log
https://neo4j.com/

In [ ]:
import json

j = json.dumps(data)
# open file for writing, "w"
f = open("neo4j_docs.json", "w")

# write json object to file
f.write(j)

# close file
f.close()

In [ ]:
# Import to Neo4j
from graphdatascience import GraphDataScience

host = "bolt://44.200.120.93:7687"
user = "neo4j"
password = "customs-spans-misalinements"

gds = GraphDataScience(host, auth=(user, password))

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Page) REQUIRE p.url IS UNIQUE;
"""
)

gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (k:Keyword) REQUIRE k.name IS UNIQUE;
"""
)

In [ ]:
import_query = """

UNWIND $data AS row
MERGE (p:Page {url:row.url})
SET p.embedding = row.embedding,
    p.has_text = row.has_text,
    p.is_404 = row.is_404
FOREACH (l in row.links    | MERGE (p1:Page {url:l}) MERGE (p)-[:LINKS_TO]->(p1))
FOREACH (k in row.keywords | MERGE (k1:Keyword {name:k}) MERGE (p)-[:HAS_KEYWORD]->(k1))
FOREACH (r in row.redirects| MERGE (r1:Page {url:r}) MERGE (p)-[:REDIRECTS]->(r1))

"""
x = 1
params = []
for key in data:
    params.append(
        {
            "url": key,
            "embedding": data[key]["embeddings"],
            "keywords": data[key]["keywords"],
            "links": data[key]["links"],
            "has_text": True if data[key]["text"] else False,
            "is_404": True if data[key]["text"] == "404" else False,
            "redirects": data[key]["redirects"]
        }
    )
    # Batch per 500
    if len(params) == 500:
        gds.run_cypher(import_query, {"data": params})
        params = []
        # Logging
        print(f"Importing {x} batch")
        x += 1

# Import the remainder
gds.run_cypher(import_query, {"data": params})